In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import skimage
import scipy
from scipy.io import loadmat

from tensorflow.keras import layers
from keras.layers import Input, Dense
from keras.models import Model
import mxnet as mx
from tensorflow import keras
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
# 定义输入层
from tensorflow.keras.models import load_model


2023-07-06 19:49:31.851991: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
data = np.load('/home/xipeng/FLcode/organamnist.npz')
data.files

x_train = data["train_images"]
y_train = data["train_labels"]
x_test = data["test_images"]
y_test = data["test_labels"]

# 为图像数据添加通道维度
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# 归一化图像数据
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0



import random

# 设备数量和IID程度
num_devices = 30
iid_ratio = 0.5

# 计算每个设备应包含的类别数量
num_classes = len(np.unique(y_train))
samples_per_device = len(x_train) // num_devices
selected_classes_per_device = int(num_classes * iid_ratio)

# 分配数据
device_data = [[] for _ in range(num_devices)]

# 为每个设备分配数据
for device_id in range(num_devices):
    # 随机选择类别
    random_classes = list(range(num_classes))
    random.shuffle(random_classes)
    selected_classes = random_classes[:selected_classes_per_device]

    # 获取已选类别的索引
    selected_indices = []
    for cls in selected_classes:
        class_indices = np.where(np.array(y_train) == cls)[0]
        selected_indices.extend(class_indices)

    # 在已选类别的索引中进行随机抽样
    random_selected_indices = random.sample(selected_indices, samples_per_device)
    device_data[device_id] = [(x_train[i], y_train[i]) for i in random_selected_indices]
y_train = keras.utils.to_categorical(y_train, num_classes)
# 检查每个设备上的样本数量
for i in range(num_devices):
    print(f"Device {i}: {len(device_data[i])} samples")
    print(f"Device {i}: ", np.unique(np.array([label for _, label in device_data[i]]), return_counts=True))  
data_x = [np.array([sample for sample, _ in device_data[i]]) for i in range(num_devices)]
data_y = [np.array([label for _, label in device_data[i]]) for i in range(num_devices)]
data_y = keras.utils.to_categorical(data_y, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = data_x
y_train = data_y

data_x_1 = data_x[0]

data_y_1 = data_y[0]

Device 0: 1152 samples
Device 0:  (array([ 2,  3,  5,  8, 10], dtype=uint8), array([114, 113, 300, 331, 294]))
Device 1: 1152 samples
Device 1:  (array([2, 6, 7, 8, 9], dtype=uint8), array([ 93, 369, 249, 243, 198]))
Device 2: 1152 samples
Device 2:  (array([1, 5, 7, 8, 9], dtype=uint8), array([115, 287, 273, 257, 220]))
Device 3: 1152 samples
Device 3:  (array([ 0,  3,  7,  8, 10], dtype=uint8), array([155, 113, 299, 328, 257]))
Device 4: 1152 samples
Device 4:  (array([3, 5, 6, 7, 9], dtype=uint8), array([ 98, 218, 399, 229, 208]))
Device 5: 1152 samples
Device 5:  (array([ 1,  4,  5,  7, 10], dtype=uint8), array([110, 278, 252, 271, 241]))
Device 6: 1152 samples
Device 6:  (array([2, 3, 5, 6, 9], dtype=uint8), array([106, 103, 290, 442, 211]))
Device 7: 1152 samples
Device 7:  (array([1, 3, 4, 8, 9], dtype=uint8), array([133, 127, 287, 341, 264]))
Device 8: 1152 samples
Device 8:  (array([0, 2, 3, 5, 9], dtype=uint8), array([192, 119, 139, 391, 311]))
Device 9: 1152 samples
Device 9

In [3]:
def add_partial_backdoor(images, labels_onehot, backdoor_label=8, trigger_size=3, backdoor_ratio=0.5):
    backdoor_images = []
    backdoor_labels_onehot = []

    num_backdoor_samples = int(len(images) * backdoor_ratio)
    backdoor_indices = np.random.choice(len(images), size=num_backdoor_samples, replace=False)

    for idx, (image, label_onehot) in enumerate(zip(images, labels_onehot)):
        backdoor_image = np.copy(image)
        backdoor_label_onehot = np.copy(label_onehot)
        
        if idx in backdoor_indices:
            backdoor_image[-trigger_size:, -trigger_size:] = 1.0
            backdoor_label_onehot = np.zeros_like(label_onehot)
            backdoor_label_onehot[backdoor_label] = 1
            
        backdoor_images.append(backdoor_image)
        backdoor_labels_onehot.append(backdoor_label_onehot)

    return np.array(backdoor_images), np.array(backdoor_labels_onehot)

In [4]:
def segmentation_data(x_train,y_train,n):   
    data_x = {}
    data_y = {}
    t = int(60000/n)
    print(t)
    print(n)
    for i in range(n):
        data_x[i] = x_train[t*i:t+t*i,:,:]

        data_y[i] = y_train[t*i:t+t*i,:]

    print(data_x[n-1].shape)
    print(data_y[n-1].shape)
    return data_x,data_y

In [5]:
def  set_model():
    model=Sequential()
        # 定义顺序模型           
    model.add(Convolution2D(
        input_shape = (28,28,1),
        filters = 8,
        kernel_size = 3,
        strides = 1,
        padding = 'same',
        activation = 'relu',
    ))
    # 第一个池化层
    model.add(MaxPooling2D(
        pool_size = 2,
        strides = 2,
        padding = 'same',
    ))
    # 第二个卷积层
    model.add(Convolution2D(16,3,strides=1,padding='same',activation='relu'))
    # 第二个池化层
    model.add(MaxPooling2D(2,2,'same'))                                    
                  
       # 第二个卷积层      
    # 把第三个池化层的输出扁平化为1维
    model.add(layers.Flatten())
    # 第一个全连接层
    model.add(Dense(256,activation='relu'))
    # Dropout
    model.add(Dropout(0.5)) 
    # 第二个全连接层
    model.add(Dense(num_classes,activation='softmax'))

    # 定义优化器
    adam = Adam(lr = 5e-4)        
        
    # 定义优化器,loss function,训练过程中计算准确率
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

        #print(model_d[i])
    return model

In [6]:
def e_x(weights):
    extracted_weights = []
    for i in range(len(weights)):
            # 卷积层的权重参数和偏置参数
        if weights[i].ndim == 4:
            conv_weights = weights[i][:, :, 0, 0]
            # 将卷积核转换为列向量
            conv_weights_col = np.reshape(conv_weights, (-1,))
            extracted_weights.append(conv_weights_col)
            # 全连接层的权重参数和偏置参数
        elif weights[i].ndim == 2:
            array = weights[i].flatten()
            extracted_weights.append(array[:20])
        elif weights[i].ndim == 1:
            if len(weights[i]) >= 20:
                extracted_weights.append(weights[i][0:20])
            else:
                extracted_weights.append(weights[i])
    return extracted_weights

In [7]:


model_test = set_model()
model_test_1 = set_model()
model_test_2 = set_model()
results = []
results2 = []
for i in range(200):
    model_test.fit(x_test,y_test,batch_size=16,epochs=1,verbose=0)
    weights = model_test.get_weights()
    extracted_vectors_1 = e_x(weights)
    result_all = np.concatenate([extracted_vectors_1[0], extracted_vectors_1[1], extracted_vectors_1[2], extracted_vectors_1[3], extracted_vectors_1[4],extracted_vectors_1[5], extracted_vectors_1[6], extracted_vectors_1[7]])               
    results.append(result_all)
matrix = np.vstack(results)

for i in range(100):
    model_test_2.fit(x_test,y_test,batch_size=16,epochs=2,verbose=0)
    weights = model_test.get_weights()
    extracted_vectors_1 = e_x(weights)
    result_all = np.concatenate([extracted_vectors_1[0], extracted_vectors_1[1], extracted_vectors_1[2], extracted_vectors_1[3], extracted_vectors_1[4],extracted_vectors_1[5], extracted_vectors_1[6], extracted_vectors_1[7]])               
    results.append(result_all)
matrix_2 = np.vstack(results)


for i in range(50):
    model_test_1.fit(x_test,y_test,batch_size=16,epochs=1,verbose=0)
    weights = model_test_1.get_weights()
    extracted_vectors_1 = e_x(weights)
    result_all2 = np.concatenate([extracted_vectors_1[0], extracted_vectors_1[1], extracted_vectors_1[2], extracted_vectors_1[3], extracted_vectors_1[4],extracted_vectors_1[5], extracted_vectors_1[6], extracted_vectors_1[7]])               
    results2.append(result_all2)
matrix2 = np.vstack(results2)



2023-07-06 19:49:38.484354: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-07-06 19:49:38.519935: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2023-07-06 19:49:38.519998: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-07-06 19:49:38.520076: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-07-06 19:49:38.520108: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-07-06 19:49:38.520141: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [8]:
def build_encoder(input_dim, hidden_units, latent_dim):
    encoder_input = Input(shape=(input_dim,))
    encoder_hidden_1 = Dense(hidden_units, activation='relu')(encoder_input)
    encoder_hidden_2 = Dense(hidden_units, activation='relu')(encoder_hidden_1)
    encoder_output = Dense(latent_dim, activation='relu')(encoder_hidden_2)
    encoder = Model(encoder_input, encoder_output)
    return encoder

def build_decoder(latent_dim, hidden_units, output_dim):
    decoder_input = Input(shape=(latent_dim,))
    decoder_hidden_1 = Dense(hidden_units, activation='relu')(decoder_input)
    decoder_hidden_2 = Dense(hidden_units, activation='relu')(decoder_hidden_1)
    decoder_output = Dense(output_dim, activation='linear')(decoder_hidden_2)
    decoder = Model(decoder_input, decoder_output)
    return decoder

def build_autoencoder(input_dim, hidden_units, latent_dim):
    encoder = build_encoder(input_dim, hidden_units, latent_dim)
    decoder = build_decoder(latent_dim, hidden_units, input_dim)

    autoencoder_input = Input(shape=(input_dim,))
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)
    autoencoder = Model(autoencoder_input, decoded)

    return autoencoder

# 设置参数
input_dim = 113
latent_dim = 100
hidden_units = 500

# 构建自编码器
autoencoder = build_autoencoder(input_dim, hidden_units, latent_dim)

# 编译模型
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

print(autoencoder.summary())

# 设置参数
input_dim = 113
latent_dim = 100
hidden_units = 500

# 构建自编码器
autoencoder = build_autoencoder(input_dim, hidden_units, latent_dim)

# 编译模型
autoencoder.compile(optimizer=Adam(learning_rate=2e-5), loss='mean_squared_error')


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 113)]             0         
_________________________________________________________________
functional_1 (Functional)    (None, 100)               357600    
_________________________________________________________________
functional_3 (Functional)    (None, 113)               357613    
Total params: 715,213
Trainable params: 715,213
Non-trainable params: 0
_________________________________________________________________
None


In [9]:

x_train_e = matrix  # 输入数据 (shape: (num_samples, input_dim))
x_test_e = matrix2   # 测试数据 (shape: (num_test_samples, input_dim))
# 训练模型
autoencoder.fit(x_train_e, x_train_e,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test_e, x_test_e))

reconstructed_test_1 = autoencoder.predict(x_train_e)

Epoch 1/50
1/1 [==============================] - 0s 254ms/step - loss: 0.0970 - val_loss: 0.0497
Epoch 2/50
1/1 [==============================] - 0s 37ms/step - loss: 0.0965 - val_loss: 0.0497
Epoch 3/50
1/1 [==============================] - 0s 41ms/step - loss: 0.0960 - val_loss: 0.0497
Epoch 4/50
1/1 [==============================] - 0s 41ms/step - loss: 0.0955 - val_loss: 0.0497
Epoch 5/50
1/1 [==============================] - 0s 34ms/step - loss: 0.0950 - val_loss: 0.0496
Epoch 6/50
1/1 [==============================] - 0s 40ms/step - loss: 0.0945 - val_loss: 0.0496
Epoch 7/50
1/1 [==============================] - 0s 35ms/step - loss: 0.0941 - val_loss: 0.0496
Epoch 8/50
1/1 [==============================] - 0s 37ms/step - loss: 0.0936 - val_loss: 0.0496
Epoch 9/50
1/1 [==============================] - 0s 37ms/step - loss: 0.0932 - val_loss: 0.0496
Epoch 10/50
1/1 [==============================] - 0s 36ms/step - loss: 0.0927 - val_loss: 0.0495
Epoch 11/50
1/1 [===========

In [10]:
#所有的客户端训练一遍
def fit_allmodel(model_d,att_k):
    
    weights_1 = {}
    for i in range(len(model_d)):
    # 训练模型
        model_d[i].fit(data_x[i],data_y[i],batch_size=16,epochs=3,verbose=0)
        loss,accuracy = model_d[i].evaluate(x_test,y_test,verbose=0)         
    for i in range(len(model_d)): 
        weights_1[i] = model_d[i].get_weights() 
    #对前5个数据进行          
    for i in range (att_k):
        for j in range(8):
            weights_1[i][j] = skimage.util.random_noise(weights_1[i][j],mode="gaussian",var = 0.2,clip=True)
        model_d[i].set_weights(weights_1[i])              
    return model_d, weights_1

In [11]:
#服务器将客户端数据集中并分配给客户端
def fed_avg(model_d,indexes_below_mean):
    weights = {}
    loss_all=[]
    acc_all=[]
 
    #这个for是得到所有模型的参数，那在这里保存所有模型的参数
    for i in range(len(model_d)): 
        weights[i] = model_d[i].get_weights() 
    index = indexes_below_mean[0] 
    last = index[len(index)-1]
 #################################################
    weights_avg = []
    weights_sum = weights[last]   #这也不能写保存最后一个了
##################################################    
    for i in range (len(weights[0])):  #  1个客户端的n层参数
        for j in range (len (index)-1): #相当于n个客户端
            j1= index[j]
            weights_sum[i] = weights_sum[i]+ weights[j1][i]   #这里出现过错误，应该是先客户端【j】再客户端的层【i】
        weights_sum[i] = weights_sum[i] /len(index)
        weights_avg.append(weights_sum[i])
    #将集合后的数据分配给每一个客户端
    for i in range (len (weights)):
        model_d[i].set_weights(weights_avg)
    #测试客户端的精度
    loss,accuracy= model_d[i].evaluate(x_test,y_test)
#################################################   
    return model_d,loss,accuracy
 #################################################

In [12]:
def FL_F (weights):
    result_50 = []
    for i in range(30):
        extracted_vectors_1 = e_x(weights[i])
        result = np.concatenate([extracted_vectors_1[0], extracted_vectors_1[1], extracted_vectors_1[2], extracted_vectors_1[3], extracted_vectors_1[4],extracted_vectors_1[5], extracted_vectors_1[6], extracted_vectors_1[7]])               
        result_50.append(result)
    matrix = np.vstack(result_50)
    recons_50 = autoencoder.predict(matrix)   
    euclidean_distances = np.sqrt(np.sum((recons_50 - matrix) ** 2, axis=1))
# 计算euclidean_distances的均值
    mean_distance = np.mean(euclidean_distances)
# 找到小于均值的所有索引
    indexes_below_mean = np.where(euclidean_distances < mean_distance)
    print(indexes_below_mean[0])
    return indexes_below_mean

In [13]:

#for i in range(15):
#    data_x[i],data_y[i] = add_partial_backdoor(data_x[i],data_y[i])

    
    
def add_backdoor_to_all(images, trigger_size=3, backdoor_label=8):
    backdoor_images = []
    backdoor_labels_onehot = []

    for image in images:
        backdoor_image = np.copy(image)
        backdoor_image[-trigger_size:, -trigger_size:] = 1.0
        backdoor_images.append(backdoor_image)

    return np.array(backdoor_images)
x_test_door = add_backdoor_to_all(x_test)
y_test_oneshot_8 = np.zeros_like(y_test)
y_test_oneshot_8[:, 8] = 1    

In [14]:
n=30
model_d = []
for i in range(30):
    model = set_model()
    model_d.append(model)
    
att_k = 9;

for s in range (100):
    model_d,weights_1 = fit_allmodel(model_d,att_k)
    
    print("***********************************************************************")
    indexes_below_mean = FL_F(weights_1)
    model_d,loss_x_y,accuracy_x_y  = fed_avg(model_d,indexes_below_mean)
# data_finall 太大，不能保存， 只能分开保存，想一下 如何一次运行能保存N个文件，试一下 程序名 +n 来进行保存

***********************************************************************
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 2.3967 - accuracy: 0.1060
***********************************************************************
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 2.1976 - accuracy: 0.2704
***********************************************************************
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.8682 - accuracy: 0.4160
***********************************************************************
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.7387 - accuracy: 0.4259
***********************************************************************
[ 9 10 11 12 13 14 15 16

In [15]:

#####需要再FL_F之前保存攻击的weight fed——all中 实现

In [16]:
n=30
model_d = []
for i in range(30):
    model = set_model()
    model_d.append(model)
    
att_k = 6;

for s in range (100):
    model_d,weights_1 = fit_allmodel(model_d,att_k)
    
    print("***********************************************************************")
    indexes_below_mean = FL_F(weights_1)
    model_d,loss_x_y,accuracy_x_y  = fed_avg(model_d,indexes_below_mean)
# data_finall 太大，不能保存， 只能分开保存，想一下 如何一次运行能保存N个文件，试一下 程序名 +n 来进行保存

***********************************************************************
[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 4ms/step - loss: 2.3970 - accuracy: 0.0583
***********************************************************************
[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 2.1892 - accuracy: 0.2801
***********************************************************************
[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.9234 - accuracy: 0.4009
***********************************************************************
[ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.7766 - accuracy: 0.4632
************************************************************

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
n=30
model_d = []
for i in range(30):
    model = set_model()
    model_d.append(model)
    
att_k = 3;

for s in range (100):
    model_d,weights_1 = fit_allmodel(model_d,att_k)
    
    print("***********************************************************************")
    indexes_below_mean = FL_F(weights_1)
    model_d,loss_x_y,accuracy_x_y  = fed_avg(model_d,indexes_below_mean)
# data_finall 太大，不能保存， 只能分开保存，想一下 如何一次运行能保存N个文件，试一下 程序名 +n 来进行保存

***********************************************************************
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 2.4002 - accuracy: 0.1273
***********************************************************************
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 2.1572 - accuracy: 0.3289
***********************************************************************
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.8613 - accuracy: 0.4412
***********************************************************************
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29]
556/556 [==============================] - 2s 3ms/step - loss: 1.6986 - accuracy: 0.4571
********************

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

